### INSTITUTO POLITÉCNICO DE LEIRIA
### Mestrado em Ciência de Dados - Projecto de Data Mining
#### Trabalho realizado por Tiago Ribeiro e Rute Santos
#### Manutenção Preditiva de uma Máquina Industrial - Parte 2


### Importacão de Bibliotecas

In [ ]:
# -- bibliotecas de uso geral --
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time
# -- pré-processamento ---------
# https://contrib.scikit-learn.org/category_encoders/ordinal.html
from category_encoders import OrdinalEncoder 
import category_encoders
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from collections import Counter 
# -- métricas ------------------ 
#from sklearn.metrics import PrecisionRecallDisplay,RocCurveDisplay
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, make_scorer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
# -- modelos -------------------
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn import tree
# -- optimizacão ---------------
from skopt import BayesSearchCV 

In [ ]:
# Leitura ficheiro do conjunto de dados 'AI4I 2020 Predictive Maintenance Dataset'
data = pd.read_csv("/work/Proj2_ai4i2020.csv") 

### Pré-processamento da variáveis

In [ ]:
# Renomeia nome das variáveis para melhor compreensão e para eliminar espacos nos nomes
data.columns = ['udi', 'product_id', 'quality', 'air_temp', 'process_temp', 
              'rotational_speed', 'torque', 'tool_wear', 'machine_failed', 
              'tool_wear_failure', 'heat_dissipation_failure', 'power_failure', 
              'overstrain_failure', 'random_failure']

# Converte graus Kelvin para Celsius nas variáveis de temperatura
data["air_temp"] = data["air_temp"].subtract(272,15)
data["process_temp"] = data["process_temp"].subtract(272,15)

In [ ]:
data

,udi,product_id,quality,air_temp,process_temp,rotational_speed,torque,tool_wear,machine_failed,tool_wear_failure,heat_dissipation_failure,power_failure,overstrain_failure,random_failure
0,1,M14860,M,26.1,36.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,26.2,36.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,26.1,36.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,26.2,36.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,26.2,36.7,1408,40.0,9,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,26.8,36.4,1604,29.5,14,0,0,0,0,0,0
9996,9997,H39410,H,26.9,36.4,1632,31.8,17,0,0,0,0,0,0
9997,9998,M24857,M,27.0,36.6,1645,33.4,22,0,0,0,0,0,0
9998,9999,H39412,H,27.0,36.7,1408,48.5,25,0,0,0,0,0,0


#### Codificacão da variável ordinal de entrada *quality*

In [ ]:
encoder = OrdinalEncoder(
    cols = ['quality'],
    return_df = True,
    mapping = [{
        'col':'quality',
        'mapping':{'L': 0, 'M': 1, 'H': 2 }}]
        )
data['quality'] = encoder.fit_transform(data['quality'])
# encoder.inverse_transform(encoded)

In [ ]:
print("Tabela de Frequência da Variável 'quality'\n")
print("'O' => Qualidade Baixa\n'1' => Qualidade Média\n'2' => Qualidade Alta\n")
print(data['quality'].value_counts())

Tabela de Frequência da Variável 'quality'

'O' => Qualidade Baixa
'1' => Qualidade Média
'2' => Qualidade Alta

0    6000
1    2997
2    1003
Name: quality, dtype: int64


#### Divisão do conjunto de dados em entradas e saídas
variáveis de **entrada**: *['quality', 'air_temp', 'process_temp', 'rotational_speed', 'torque', 'tool_wear']* <br>
variáveis de **saída** ou objetivo: *['machine_failed', 'tool_wear_failure', 'heat_dissipation_failure', 'power_failure' 'overstrain_failure', 'random_failure']*

In [ ]:
entrada = data[['quality', 'air_temp', 'process_temp', 'rotational_speed', 'torque', 'tool_wear']]
saida = data[['machine_failed','tool_wear_failure','heat_dissipation_failure','power_failure', 
              'overstrain_failure', 'random_failure']]

In [ ]:
saida

,machine_failed,tool_wear_failure,heat_dissipation_failure,power_failure,overstrain_failure,random_failure
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
9995,0,0,0,0,0,0
9996,0,0,0,0,0,0
9997,0,0,0,0,0,0
9998,0,0,0,0,0,0


#### Normalizacão por Padronizacão das variáveis de entrada
A cada valor do atributo é subtraída a média da amostra e dividido pelo desvio padrão. Depois deste processo, as amostras transformadas ficam com uma distribuicão aproxidamente normal, com média igual a 0 e desvio padrão unitário: <b>N~(u=0, s=1)</b>.

In [ ]:
padronizar = True # 'False' para Arvores de Decisão, apenas
if padronizar == True:
    # define padronizador => https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
    padronizador = StandardScaler()
    # padroniza os dados 
    entrada = padronizador.fit_transform(entrada)
    # guarda padronizador em ficheiro
    pickle.dump(padronizador, open('padronizador.sav', 'wb'))
print('Padronizacão: ' + str(padronizar))

Padronizacão: True


#### Codificão das Classes
Os valores binários discretos são concatenados e tranformados em valores numéricos inteiros.
No total, o conjunto de dados tem 13 categorias: sendo que a primeira - 000000<sub>2</sub> ou 0<sub>10</sub> - indica que não houve falha na máquina e que tem valores diferente de 0 indicam um qualquer tipo de falhas ou combinacão de falhas.

In [ ]:
# converte vetor binário num número inteiro
# https://numpy.org/doc/stable/reference/generated/numpy.unpackbits.html
saida_int = np.packbits(saida, bitorder='little', axis=1)
# codifica rótulos de y com valor entre 0 e n_classes-1
label_encoder = LabelEncoder()
saida_cod = label_encoder.fit_transform(saida_int)

print('Classes codificadas\n Decimal: ' + str(list(label_encoder.classes_)))
classes_bin = []
for i in range(0, len(label_encoder.classes_)):
     classes_bin.append(np.binary_repr(label_encoder.classes_[i], width = 6))
print(' Binário: '+ str(list(classes_bin))+'\n')

Classes codificadas
 Decimal: [0, 1, 3, 5, 9, 13, 17, 19, 21, 25, 27, 32, 35]
 Binário: ['000000', '000001', '000011', '000101', '001001', '001101', '010001', '010011', '010101', '011001', '011011', '100000', '100011']

/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#### Re-Amostragem dos Dados com *RandomOverSampler*
Com o propósito de baleancear o conjunto de dados utilizou o algoritmo ROSE (**R**andom **O**ver **S**ample **E**xamples), implementado pela funcão *RandomOverSampler* 

Ref: Training and assessing classification rules
with imbalanced data, http://cloud.politala.ac.id/politala/Jurnal/JurnalTI/Jurnal%2035/2Fs10618-012-0295-5.pdf





In [ ]:
# semente para os geradores de números aleatórios
semente = 42

In [ ]:
print('Número de ocorrências por categoria original\n%s' % Counter(saida_cod))
amostrador = RandomOverSampler(random_state = semente)
entrada_res, saida_cod_res = amostrador.fit_resample(entrada, saida_cod)
print('Número de ocorrências por categoria após re-amostragem\n%s' % Counter(saida_cod_res))

Número de ocorrências por categoria original
Counter({0: 9643, 3: 106, 4: 80, 6: 78, 2: 42, 11: 18, 9: 11, 1: 9, 8: 6, 5: 3, 7: 2, 12: 1, 10: 1})
Número de ocorrências por categoria após re-amostragem
Counter({0: 9643, 4: 9643, 9: 9643, 2: 9643, 6: 9643, 11: 9643, 1: 9643, 3: 9643, 12: 9643, 5: 9643, 8: 9643, 7: 9643, 10: 9643})


#### Divisão do conjunto de Dados

<table>
<thead>
  <tr>
    <th colspan="3"> </th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>Treino/Validacão </td>
    <td>| 9000 linhas </td>
    <td>| 90 % </td>
  </tr>
  <tr>
    <td>Teste </td>
    <td>| 1000 linhas  <br></td>
    <td>| 10 % </td>
  </tr>
  <tr>
    <td><strong>Total </strong></td>
    <td>| 10000 linhas </td>
    <td>|<strong> 100 % </strong> </td>
  </tr>
</tbody>
</table>




  



![Picture title](image-20220720-014234.png)

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(
    entrada_res,saida_cod_res,     # variáveis re-amostradas 
    #entrada,saida_cod,            # variávei não re-amostradas
    test_size = 0.10, 
    random_state = semente, 
    shuffle = True)

### Funcões várias 

#### Funcão para calcular medidas de desempenho dos modelos 
=> Taxa de Acerto (ou Acurácia), Precisão, Sensibilidade e Medida-F



In [ ]:
def calcula_metricas(y_valid, y_prev):
    # Calcula Acurácia
    acu = accuracy_score(y_valid, y_prev)
    # Calcula Precisão
    prec = precision_score(y_valid, y_prev, average = 'macro', zero_division=1)
    # Calcula Sensibilidade
    sens = recall_score(y_valid, y_prev, average = 'macro', zero_division=1)
    # Calcula Medida-F
    f1 = f1_score(y_valid, y_prev, average = 'macro', zero_division=1)

    print('Acurácia: '+ str(round(acu*100,2)) + ' %')
    print('Com Média Não Ponderada:\nPrecisão: '+ str(round(prec*100, 2)) + ' %')
    print('\033[1mSensibilidade\033[0m: '+ str(round(sens*100,2)) + ' %')
    print('Medida-F: '+ str(round(f1*100,2)) + ' %')


#report = classification_report(y_valid, y_prev)
#print(report)
#from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(y_valid_parte, y_prev)

#### Funcão para guardar modelo num ficheiro

In [ ]:
def salva_modelo(modelo, nome_modelo): 
    # guarda modelo no disco
    filename = nome_modelo + '.sav'
    pickle.dump(modelo, open(filename, 'wb'))

#### Funcão para carregar modelos guardados em ficheiro

In [ ]:
def carrega_modelo(nome_modelo): 
    # carrega modelo do disco
    modelo_carregado = pickle.load(open(nome_modelo + '.sav', 'rb'))
    
    # escreve métricas de desempenho do modelo carregado
    y_prev = modelo_carregado.predict(x_teste)
    print((nome_modelo).upper())
    calcula_metricas(y_teste, y_prev)
    
    return modelo_carregado

### Modelos testados
#### 1. Árvores de Decisão

In [ ]:
# 1. Arvores de Decisão

modelo = DecisionTreeClassifier(
    random_state = semente, 
    min_samples_split = 2,
    max_leaf_nodes = 13
    )

# dicionário com espaco de busca de hiperparâmetros
espaco_busca  = {
    'criterion' : ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'min_samples_leaf': (1,10000),
    'max_depth' : (2,5),
}

# sensibilidade como classificador
classificador = make_scorer(recall_score, average = 'macro',  zero_division = 1) 

# optimizador bayesiano
optimizador = BayesSearchCV(
    estimator = modelo,
    search_spaces = espaco_busca,
    scoring = classificador,
    cv = 3,                         # Validação Cruzada Estratificada, particões = 3
    random_state = semente,
    verbose = 10
 )

optimizador.fit(x_treino, y_treino)

# Escreve os melhores resultados e melhores hiperparâmetros
print("Hiperparâmetros do melhor modelo")
print(optimizador.best_params_)

#### Construcão e análise desempenho do modelo com hiperparâmetros encontrados 

Com conjunto de dados original

In [ ]:
# Hiperparâmetros do melhor modelo com Re-Amostragem de Dados
# OrderedDict([('criterion', 'gini'), ('max_depth', 8), ('max_leaf_nodes', 10), ('min_samples_leaf', 10), ('splitter', 'best')])

modelo = DecisionTreeClassifier(
    criterion = 'gini', 
    max_depth = 8, 
    max_leaf_nodes = 10, 
    min_samples_leaf = 10,
    splitter = 'best'
    )

modelo.fit(x_treino, y_treino)
y_prev = modelo.predict(x_treino)
calcula_metricas(y_treino, y_prev)
salva_modelo(modelo, 'arv_dec')

# Acurácia: 97.8 %
# Média Não Ponderada
# Precisão: 96.76 %
# Sensibilidade: 21.01 %
# Medida-F: 21.12 %

Acurácia: 97.8 %
Média Não Ponderada
Precisão: 96.76 %
Sensibilidade: 21.01 %
Medida-F: 21.12 %


Com conjunto de dados reamostrados

In [ ]:
# Hiperparâmetros do melhor modelo sem Re-Amostragem de Dados
#('criterion', 'gini'), ('max_depth', 5), ('min_samples_leaf', 1049), ('splitter', 'best')])


modelo = DecisionTreeClassifier(
    criterion = 'gini', 
    max_depth = 5,  
    min_samples_leaf = 1049,
    splitter = 'best'
    )

modelo.fit(x_treino, y_treino)
y_prev = modelo.predict(x_treino)
calcula_metricas(y_treino, y_prev)
salva_modelo(modelo, 'arv_dec_res')

# Acurácia: 71.21 %
# Média Não Ponderada
# Precisão: 81.51 %
# Sensibilidade: 71.22 %
# Medida-F: 66.64 %

Acurácia: 71.21 %
Média Não Ponderada
Precisão: 81.51 %
Sensibilidade: 71.22 %
Medida-F: 66.64 %


#### 2. Máquinas de Suporte de Vetores 
A busca dos hiperâmetros quando o *kernel* é polinomial e com grau de liberdade > 2 revelou-se lenta (na ordem das horas). Por essa razão, pelo tempo e poder computacional limitado, rejeitou-se o *kernel* polinomial.


In [ ]:
# criacão de modelo de base
modelo = svm.SVC()

# espaco de busca dos hiperparâmetros
espaco_busca  = {
    'kernel': ['linear',  'rbf', 'sigmoid'],
    'C': (1, 70),
    'gamma': (0, 70),
    'degree': (1, 3)
    }

# sensibilidade como classificador
classificador = make_scorer(recall_score, average = 'macro',  zero_division = 1) 

# optimizador bayesiano
optimizador = BayesSearchCV(
    estimator = modelo,
    search_spaces = espaco_busca,
    scoring = classificador,
    cv = 3,                         # StratifiedKFold, k = 3
    random_state = semente,
    verbose = 10
 )

# treina no espaco de busca com procura bayesiana
optimizador.fit(x_treino, y_treino)

# escreve o modelo com melhores hiperparâmetros
print("Hiperparâmetros do melhor modelo")
print(optimizador.best_params_)

#### Construcão e análise desempenho do modelo com hiperparâmetros encontrados 

Com conjunto de dados original

In [ ]:
#---------------------------------------------------------------------------------
# Hiperparâmetros do melhor modelo SVM sem Re-amostragem de Dados
# ('C', 47), ('degree', 1), ('gamma', 16.082807397859792), ('kernel', 'linear')])
#---------------------------------------------------------------------------------



# cria modelo com os melhores hiperparâmetros encontrados
modelo = svm.SVC(C = 47, degree = 1, kernel= 'linear', gamma = 16.082807397859792)

# treina modelo
modelo.fit(x_treino, y_treino)
# valores previstos pelo modelo
y_prev = modelo.predict(x_treino)
# calcula e escreve métricas do modelo
calcula_metricas(y_treino, y_prev)
# guarda modelo
salva_modelo(modelo, 'svm')

#-- Resultados --------
# Acurácia: 97.55 %
# Média Não Ponderada
# Precisão: 94.75 %
# Sensibilidade: 25.86 %
# Medida-F: 25.42 %

Acurácia: 97.64 %
Média Não Ponderada
Precisão: 96.54 %
Sensibilidade: 28.1 %
Medida-F: 28.86 %


Com conjunto de dados reamostrados

In [ ]:
# Hiperparâmetros do melhor modelo com Re-amostragem de Dados
# ('C', 40), ('degree', 1), ('gamma', 2), ('kernel', 'rbf')

# cria modelo com os melhores hiperparâmetros encontrados
modelo = svm.SVC(C = 40, degree = 1, kernel= 'rbf', gamma = 2)

# treina modelo
modelo.fit(x_treino, y_treino)
# valores previstos pelo modelo
y_prev = modelo.predict(x_treino)
# calcula e escreve métricas do modelo
print('Acabou treino')
calcula_metricas(y_treino, y_prev)
# guarda modelo
salva_modelo(modelo, 'svm_res')

#-- Resultados --------
# Acurácia: 99.89 %
# Com Média Não Ponderada:
# Precisão: 99.89 %
# Sensibilidade: 99.89 %
# Medida-F: 99.89 %

Acabou treino
Acurácia: 99.89 %
Com Média Não Ponderada:
Precisão: 99.89 %
Sensibilidade: 99.89 %
Medida-F: 99.89 %


#### 3. Floresta Aleatória (Comité)


In [ ]:
# criacão de modelo de base
modelo = RandomForestClassifier()
# espaco de busca dos hiperparâmetros
espaco_busca = {
    "max_depth": (3, 10),
    "min_samples_split": (2,10), 
    "min_samples_leaf": (1,10), 
    "bootstrap": [True, False],
    "criterion": ['gini', 'entropy', 'log_loss']
   }

# sensibilidade como classificador
classificador = make_scorer(recall_score, average = 'macro',  zero_division = 1) 
# optimizador bayesiano
optimizador = BayesSearchCV(
    estimator = modelo,
    search_spaces = espaco_busca,
    scoring = classificador,
    cv = 3,                         # StratifiedKFold, k = 3
    random_state = semente,
    verbose = 1
 )

# treina no espaco de busca com procura bayesiana
optimizador.fit(x_treino, y_treino)

# Escreve os melhores hiperparâmetros
print("Hiperparâmetros do melhor modelo")
print(optimizador.best_params_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

#### Construcão e análise desempenho do modelo com hiperparâmetros encontrados 

Com conjunto de dados original

In [ ]:
# Hiperparâmetros do melhor modelo sem Re-Amostragem de Dados
# OrderedDict([('bootstrap', False), ('criterion', 'entropy'), ('max_depth', 10), ('min_samples_leaf', 1), 
# ('min_samples_split', 2)])

modelo = RandomForestClassifier(
    bootstrap = False, 
    criterion = 'entropy', 
    max_depth = 10, 
    min_samples_leaf = 1
    )

# treina modelo
modelo.fit(x_treino, y_treino)
# valores previstos pelo modelo
y_prev = modelo.predict(x_treino)
# calcula e escreve métricas do modelo
calcula_metricas(y_treino, y_prev)
# guarda modelo
salva_modelo(modelo, 'flo_ale')

# Acurácia: 98.89 %
# Média Não Ponderada
# Precisão: 99.7 %
# Sensibilidade: 66.61 %
# Medida-F: 70.74 %

Acurácia: 98.99 %
Média Não Ponderada
Precisão: 99.66 %
Sensibilidade: 64.04 %
Medida-F: 69.1 %


Com conjunto de dados reamostrados

In [ ]:
# Hiperparâmetros do melhor modelo com Re-Amostragem de Dados
# OrderedDict([('bootstrap', False), ('criterion', 'entropy'), ('max_depth', 10), ('min_samples_leaf', 1), ('min_samples_split', 2)])

modelo = RandomForestClassifier(
    bootstrap = False, 
    criterion = 'entropy', 
    max_depth = 10, 
    min_samples_leaf = 1,
    min_samples_split = 10)

# treina modelo
modelo.fit(x_treino, y_treino)
# valores previstos pelo modelo
y_prev = modelo.predict(x_treino)
# calcula e escreve métricas do modelo
calcula_metricas(y_treino, y_prev)
# guarda modelo
salva_modelo(modelo, 'flo_ale_res')

# Acurácia: 99.07 %
# Média Não Ponderada
# Precisão: 99.75 %
# Sensibilidade: 67.87 %
# Medida-F: 72.61 %

Acurácia: 99.41 %
Média Não Ponderada
Precisão: 99.43 %
Sensibilidade: 99.4 %
Medida-F: 99.4 %


#### 4. Rede Neuronal 

In [ ]:
modelo = MLPClassifier(
    solver='adam',
    activation='relu',
    verbose = 10,
    max_iter = 500,               # número de epochs máximo
    n_iter_no_change = 50,
    hidden_layer_sizes = (50,40,25)
)

espaco_busca = {
    'learning_rate_init' : (1e-6, 1.0, 'log-uniform'),
    'beta_1': (1e-6, 0.99, 'log-uniform'),
    'beta_2': (1e-6, 0.99, 'log-uniform')
}

# sensibilidade como classificador
classificador = make_scorer(recall_score, average = 'macro',  zero_division = 1) 
# optimizador bayesiano
optimizador = BayesSearchCV(
    estimator = modelo,
    search_spaces = espaco_busca,
    scoring = classificador,
    cv = 3,                         # StratifiedKFold, k = 3
    random_state = semente,
    verbose = 10
 )

# treina no espaco de busca com procura bayesiana
optimizador.fit(x_treino, y_treino)

# Escreve os melhores hiperparâmetros
print("Hiperparâmetros do melhor modelo")
print(optimizador.best_params_)

Iteration 9, loss = 2.31887935
Iteration 10, loss = 2.30314525
Iteration 11, loss = 2.28722442
Iteration 12, loss = 2.27164945
Iteration 13, loss = 2.25547696
Iteration 14, loss = 2.23952663
Iteration 15, loss = 2.22304599
Iteration 16, loss = 2.20674392
Iteration 17, loss = 2.19084327
Iteration 18, loss = 2.17501225
Iteration 19, loss = 2.15968958
Iteration 20, loss = 2.14324608
Iteration 21, loss = 2.12716424
Iteration 22, loss = 2.11169486
Iteration 23, loss = 2.09599909
Iteration 24, loss = 2.08030507
Iteration 25, loss = 2.06467902
Iteration 26, loss = 2.04888440
Iteration 27, loss = 2.03267625
Iteration 28, loss = 2.01597250
Iteration 29, loss = 1.99961463
Iteration 30, loss = 1.98303163
Iteration 31, loss = 1.95296359
Iteration 32, loss = 1.93311344
Iteration 33, loss = 1.91640546
Iteration 34, loss = 1.89951314
Iteration 35, loss = 1.88237308
Iteration 36, loss = 1.86487763
Iteration 37, loss = 1.84730882
Iteration 38, loss = 1.82942090
Iteration 39, loss = 1.81126383
Iteration

#### Construcão e análise desempenho do modelo com hiperparâmetros encontrados 

Com conjunto de dados original

In [ ]:
# Hiperparâmetros do melhor modelo sem Re-Amostragem de Dados
modelo = MLPClassifier(
    hidden_layer_sizes = (50,40,25), 
    learning_rate_init = 1e-3,
    solver='adam',
    activation = 'relu',
    verbose = 10,
    max_iter = 20000,
    n_iter_no_change = 20000)

# treina modelo
modelo.fit(x_treino, y_treino)
# valores previstos pelo modelo
y_prev = modelo.predict(x_treino)
# calcula e escreve métricas do modelo
calcula_metricas(y_treino, y_prev)
# guarda modelo
salva_modelo(modelo, 'RNA')

Iteration 19009, loss = 0.01699452
Iteration 19010, loss = 0.00912249
Iteration 19011, loss = 0.00965687
Iteration 19012, loss = 0.00823451
Iteration 19013, loss = 0.00560626
Iteration 19014, loss = 0.00462327
Iteration 19015, loss = 0.00344665
Iteration 19016, loss = 0.00220679
Iteration 19017, loss = 0.00218856
Iteration 19018, loss = 0.00203932
Iteration 19019, loss = 0.00304169
Iteration 19020, loss = 0.00302818
Iteration 19021, loss = 0.00291288
Iteration 19022, loss = 0.00342687
Iteration 19023, loss = 0.00416768
Iteration 19024, loss = 0.00353778
Iteration 19025, loss = 0.00298730
Iteration 19026, loss = 0.00237231
Iteration 19027, loss = 0.00227983
Iteration 19028, loss = 0.00239218
Iteration 19029, loss = 0.00226749
Iteration 19030, loss = 0.00238152
Iteration 19031, loss = 0.00273940
Iteration 19032, loss = 0.00197155
Iteration 19033, loss = 0.00232135
Iteration 19034, loss = 0.00529798
Iteration 19035, loss = 0.00842367
Iteration 19036, loss = 0.00431096
Iteration 19037, los

Com conjunto de dados reamostrados

In [ ]:
# Hiperparâmetros do melhor modelo com Re-Amostragem de Dados
modelo = MLPClassifier(
    hidden_layer_sizes = (50,40,25), 
    learning_rate_init = 9.984412631544676e-05,
    beta_1 = 3.885425431899516e-05,
    solver='adam',
    activation = 'relu',
    verbose = 10,
    max_iter = 1000,
    n_iter_no_change = 500)

# treina modelo
modelo.fit(x_treino, y_treino)
# valores previstos pelo modelo
y_prev = modelo.predict(x_treino)
# calcula e escreve métricas do modelo
calcula_metricas(y_treino, y_prev)
# guarda modelo
salva_modelo(modelo, 'RNA_res')

Iteration 9, loss = 0.49134277
Iteration 10, loss = 0.45515604
Iteration 11, loss = 0.42319141
Iteration 12, loss = 0.39550865
Iteration 13, loss = 0.37112847
Iteration 14, loss = 0.34928780
Iteration 15, loss = 0.32967666
Iteration 16, loss = 0.31174679
Iteration 17, loss = 0.29565337
Iteration 18, loss = 0.28163944
Iteration 19, loss = 0.26903224
Iteration 20, loss = 0.25757490
Iteration 21, loss = 0.24724229
Iteration 22, loss = 0.23787535
Iteration 23, loss = 0.22916704
Iteration 24, loss = 0.21971764
Iteration 25, loss = 0.21167017
Iteration 26, loss = 0.20452011
Iteration 27, loss = 0.19775133
Iteration 28, loss = 0.19143160
Iteration 29, loss = 0.18536225
Iteration 30, loss = 0.17984762
Iteration 31, loss = 0.17460877
Iteration 32, loss = 0.16974905
Iteration 33, loss = 0.16520389
Iteration 34, loss = 0.16082315
Iteration 35, loss = 0.15661945
Iteration 36, loss = 0.15269648
Iteration 37, loss = 0.14890593
Iteration 38, loss = 0.14542588
Iteration 39, loss = 0.14199272
Iteration

In [ ]:
# Hiperparâmetros dos varios modelos testados (iteracões) sem Re-Amostragem de Dados
#-- Modelo 1 ------------------------
# modelo = MLPClassifier(
#     hidden_layer_sizes = (15,15), 
#     learning_rate = 'constant', 
#     learning_rate_init = 3e-4,
#     solver='adam',
#     activation='relu',
#     verbose = 10,
#     max_iter = 20000,
#     n_iter_no_change = 20000)
#-- Resultados ---------------------
# Acurácia: 98.46 %
# Média Não Ponderada
# Precisão: 91.15 %
# Sensibilidade: 60.68 %
# Medida-F: 61.86 %
#-----------------------------------

#-- Modelo 2 -----------------------
# modelo = MLPClassifier(
#     hidden_layer_sizes = (15,15), 
#     learning_rate = 'constant',
#     solver='sgd', 
#     learning_rate_init = 1,
#     momentum = 0.048949234227784445,
#     activation = 'relu',
#     verbose = 10,
#     max_iter = 20000,
#     n_iter_no_change = 20000)
#-- Resultados ---------------------
# Acurácia: 98.27 %
# Média Não Ponderada
# Precisão: 87.64 %
# Sensibilidade: 49.92 %
# Medida-F: 50.83 %
#-----------------------------------

#-- Modelo 3 -----------------------
# modelo = MLPClassifier(
#     hidden_layer_sizes = (30,15), 
#     learning_rate = 'adaptive',
#     solver='sgd', 
#     learning_rate_init = 0.8,
#     momentum = 0.04,
#     activation = 'relu',
#     verbose = 10,
#     max_iter = 20000,
#     n_iter_no_change = 20000)
#-- Resultados ---------------------
# Acurácia: 98.27 %
# Média Não Ponderada
# Precisão: 69.28 %
# Sensibilidade: 58.58 %
# Medida-F: 56.89 %
#-----------------------------------

#-- Modelo 4 -----------------------
# modelo = MLPClassifier(
#     hidden_layer_sizes = (50,40,25), 
#     learning_rate_init = 1e-3,
#     solver='adam',
#     activation = 'relu',
#     verbose = 10,
#     max_iter = 20000,
#     n_iter_no_change = 20000)
#-- Resultados --------------------
# Acurácia: 99.81 %
# Média Não Ponderada
# Precisão: 98.92 %
# Sensibilidade: 96.0 %
# Medida-F: 97.15 %
#----------------------------------

# Hiperparâmetros do melhor modelo com Re-Amostragem de Dados
#-- Modelo 1 -----------------------
# modelo = MLPClassifier(
#     hidden_layer_sizes = (50,40,25), 
#     learning_rate_init = 9.984412631544676e-05,
#     beta_1 = 3.885425431899516e-05,
#     solver='adam',
#     activation = 'relu',
#     verbose = 10,
#     max_iter = 1000,
#     n_iter_no_change = 500)
#-- Resultados --------------------


### Avaliacão dos algoritmos

Análise do Desempenho do Algoritmos com o conjunto de Teste

#### Máquina de Suporte de Vetores 
Dados Reamostrados e Padronizados

In [ ]:
# teste com dados padronizados
modelo_SVM_res = carrega_modelo('svm_res')

SVM_RES
Acurácia: 99.78 %
Com Média Não Ponderada:
Precisão: 99.77 %
Sensibilidade: 99.79 %
Medida-F: 99.78 %


#### Floresta Aleatória
Dados Reamostrados e Padronizados

In [ ]:
modelo_FLO_ALE_res = carrega_modelo('flo_ale_res')

FLO_ALE_RES
Acurácia: 99.39 %
Com Média Não Ponderada:
Precisão: 99.39 %
Sensibilidade: 99.42 %
Medida-F: 99.39 %


#### Rede Neuronal 
Dados Reamostrados e Padronizados

In [ ]:
modelo_RNA_res = carrega_modelo('RNA_res')

RNA_RES
Acurácia: 99.66 %
Com Média Não Ponderada:
Precisão: 99.66 %
Sensibilidade: 99.68 %
Medida-F: 99.66 %


#### Arvore de Decisão

Dados Reamostrados e **sem Padronizacão**

    Notas para interpretacão do diagrama:

    1. Na bifurcacão do ramo, para a esquerda quando a condicão é verdadeira, para a direita quando é falsa.

    2. A categoria que apresenta maior ocorrências no fim do ramo (leaf ou folha) é a previsão que a árvore gerou.


In [ ]:
modelo_arvore = carrega_modelo('arv_dec_res')

previsores = ['quality', 'air_temp', 'process_temp', 
              'rotational_speed', 'torque', 'tool_wear']

figura, eixos = plt.subplots(nrows=1, ncols=1, figsize=(200,200))
tree.plot_tree(
    modelo_arvore, 
    feature_names = previsores, 
    filled=True
    )

ARV_DEC_RES
Acurácia: 70.96 %
Média Não Ponderada
Precisão: 81.47 %
Sensibilidade: 70.85 %
Medida-F: 66.34 %


[Text(0.590625, 0.9166666666666666, 'torque <= 65.25\ngini = 0.923\nsamples = 112823\nvalue = [8640, 8632, 8725, 8667, 8673, 8667, 8708, 8666, 8664\n8657, 8693, 8742, 8689]'),
 Text(0.31875, 0.75, 'torque <= 49.75\ngini = 0.903\nsamples = 82312\nvalue = [8629, 8632, 8725, 8582, 5251, 0, 8152, 8666, 5842\n2402, 0, 8742, 8689]'),
 Text(0.1375, 0.5833333333333334, 'rotational_speed <= 1405.5\ngini = 0.844\nsamples = 45362\nvalue = [7358, 7699, 8725, 3253, 3273, 0, 575, 0, 0, 0, 0\n5790, 8689]'),
 Text(0.075, 0.4166666666666667, 'rotational_speed <= 1404.5\ngini = 0.592\nsamples = 14771\nvalue = [629, 0, 1476, 3253, 0, 0, 211, 0, 0, 0, 0, 513\n8689]'),
 Text(0.05, 0.25, 'air_temp <= 29.55\ngini = 0.635\nsamples = 6070\nvalue = [617, 0, 1476, 3253, 0, 0, 211, 0, 0, 0, 0, 513\n0]'),
 Text(0.025, 0.08333333333333333, 'gini = 0.622\nsamples = 2683\nvalue = [483, 0, 1476, 0, 0, 0, 211, 0, 0, 0, 0, 513, 0]'),
 Text(0.075, 0.08333333333333333, 'gini = 0.076\nsamples = 3387\nvalue = [134, 0, 0, 32

#### Tempo de Inferência

Contabilizacão do tempo que os algorimos levam para calcular a previsão de todos os dados de treino.

Arvore de Decisão

In [ ]:
start_time = time.time()
y_prev = modelo_arvore.predict(x_treino)
print(print("--- %s segundos ---" % (time.time() - start_time)))

--- 0.019882678985595703 segundos ---
None
/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Máquina de Suporte de Vetores

In [ ]:
start_time = time.time()
y_prev = modelo_SVM_res.predict(x_treino)
print(print("--- %s segundos ---" % (time.time() - start_time)))

--- 44.61881709098816 segundos ---
None


Rede Neuronal

In [ ]:
start_time = time.time()
y_prev = modelo_RNA_res.predict(x_treino)
print(print("--- %s segundos ---" % (time.time() - start_time)))

--- 0.8697628974914551 segundos ---
None


In [ ]:
Floresta Aleatória

In [ ]:
start_time = time.time()
y_prev = modelo_FLO_ALE_res.predict(x_treino)
print(print("--- %s segundos ---" % (time.time() - start_time)))

--- 2.3814215660095215 segundos ---
None


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=949f6d12-6858-45c7-af2b-de7c12d6253d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>